In [ ]:
import pickle
import pandas as pd
from typing import Callable, Optional
import torch
import os
import einops
import matplotlib.pyplot as plt

import circuits.eval_sae_as_classifier as eval_sae
import circuits.analysis as analysis
import circuits.eval_board_reconstruction as eval_board_reconstruction
import circuits.get_eval_results as get_eval_results
import circuits.f1_analysis as f1_analysis
import circuits.utils as utils
import circuits.pipeline_config as pipeline_config

In [ ]:
device = torch.device("cuda:0")

autoencoder_group_path = "../autoencoders/testing_chess/"
autoencoder_path = "../autoencoders/testing_chess/trainer4/"

othello = eval_sae.check_if_autoencoder_is_othello(autoencoder_group_path)
config = pipeline_config.Config()

# These both significantly reduce peak GPU memory usage
config.batch_size = 5
config.analysis_on_cpu = True

# Precompute will create both datasets and save them as pickle files
# If precompute == False, it creates the dataset on the fly
# This is far slower when evaluating multiple SAEs, but for an exploratory run it is fine
config.precompute = False

config.eval_results_n_inputs = 500
config.eval_sae_n_inputs = 500
config.board_reconstruction_n_inputs = 500

# Once you have ran the analysis, you can set this to False and it will load the saved results
config.run_analysis = True
config.run_board_reconstruction = True
config.run_eval_sae = False
config.run_eval_results = True

# If you want to save the results of the analysis
config.save_results = True
config.save_feature_labels = True

print(f"Is Othello: {othello}")

In [ ]:
dataset_size = max(config.eval_sae_n_inputs, config.board_reconstruction_n_inputs)

# We have plenty of data and eval_results_data doesn't use VRAM, so we can afford to make it large
# So we don't hit the end of the activation buffer
eval_results_dataset_size = config.eval_results_n_inputs * 10

indexing_functions = eval_sae.get_recommended_indexing_functions(othello)
indexing_function = indexing_functions[0]

if othello:
    custom_functions = config.othello_functions
    game_name = "othello"
else:
    custom_functions = config.chess_functions
    game_name = "chess"

train_dataset_name = f"{game_name}_train_dataset.pkl"
test_dataset_name = f"{game_name}_test_dataset.pkl"

if os.path.exists(train_dataset_name) and config.precompute:
    print("Loading statistics aggregation dataset")
    with open(train_dataset_name, "rb") as f:
        train_data = pickle.load(f)
else:
    print("Constructing statistics aggregation dataset")
    train_data = eval_sae.construct_dataset(
        othello,
        custom_functions,
        dataset_size,
        split="train",
        device=device,
        precompute_dataset=config.precompute,
    )
    if config.precompute:
        print("Saving statistics aggregation dataset")
        with open(train_dataset_name, "wb") as f:
            pickle.dump(train_data, f)

if os.path.exists(test_dataset_name) and config.precompute:
    print("Loading test dataset")
    with open(test_dataset_name, "rb") as f:
        test_data = pickle.load(f)
else:
    print("Constructing test dataset")
    test_data = eval_sae.construct_dataset(
        othello,
        custom_functions,
        dataset_size,
        split="test",
        device=device,
        precompute_dataset=config.precompute,
    )
    if config.precompute:
        print("Saving test dataset")
        with open(test_dataset_name, "wb") as f:
            pickle.dump(test_data, f)

eval_results_data = eval_sae.construct_dataset(
    othello,
    [],
    eval_results_dataset_size,
    split="train",
    device=device,
    precompute_dataset=config.precompute,
)

In [ ]:
expected_eval_results_output_location = get_eval_results.get_output_location(
    autoencoder_path, n_inputs=config.eval_results_n_inputs
)

if config.run_eval_results:

    # If this is set, everything below should be reproducible
    # Then we can just save results from 1 run, make optimizations, and check that the results are the same
    # The determinism is only needed for getting activations from the activation buffer for finding alive features
    torch.manual_seed(0)
    eval_results = get_eval_results.get_evals(
        autoencoder_path,
        config.eval_results_n_inputs,
        config.batch_size,
        device,
        utils.to_device(eval_results_data.copy(), device),
        othello=othello,
        save_results=config.save_results,
    )
else:
    with open(expected_eval_results_output_location, "rb") as f:
        eval_results = pickle.load(f)
    eval_results = utils.to_device(eval_results, device)

In [ ]:
print(f"L0: {eval_results['eval_results']['l0']}")
print(f"Loss recovered: {eval_results['eval_results']['frac_recovered']}")

In [ ]:
expected_aggregation_output_location = eval_sae.get_output_location(
    autoencoder_path,
    n_inputs=config.eval_sae_n_inputs,
    indexing_function=indexing_function,
)

if config.run_eval_sae:
    print("Aggregating", autoencoder_path)
    aggregation_results = eval_sae.aggregate_statistics(
        custom_functions=custom_functions,
        autoencoder_path=autoencoder_path,
        n_inputs=config.eval_sae_n_inputs,
        batch_size=config.batch_size,
        device=device,
        data=utils.to_device(train_data.copy(), device),
        thresholds_T=config.f1_analysis_thresholds,
        indexing_function=indexing_function,
        othello=othello,
        save_results=config.save_results,
        precomputed=config.precompute,
    )
else:
    with open(expected_aggregation_output_location, "rb") as f:
        aggregation_results = pickle.load(f)
    aggregation_results = utils.to_device(aggregation_results, device)

In [ ]:
if config.analysis_on_cpu:
    aggregation_results = utils.to_device(aggregation_results, "cpu")
    analysis_device = "cpu"
else:
    analysis_device = device

torch.cuda.empty_cache()

expected_feature_labels_output_location = expected_aggregation_output_location.replace(
    "results.pkl", "feature_labels.pkl"
)
if config.run_analysis:
    feature_labels, misc_stats = analysis.analyze_results_dict(
        aggregation_results,
        output_path=expected_feature_labels_output_location,
        device=analysis_device,
        high_threshold=config.analysis_high_threshold,
        low_threshold=config.analysis_low_threshold,
        significance_threshold=config.analysis_significance_threshold,
        verbose=False,
        print_results=False,
        save_results=config.save_feature_labels,
    )
else:
    with open(expected_feature_labels_output_location, "rb") as f:
        feature_labels = pickle.load(f)
    feature_labels = utils.to_device(feature_labels, analysis_device)


In [ ]:
def rc_to_square_notation(row, col):
    letters = "ABCDEFGH"
    number = row + 1
    letter = letters[col]
    return f"{letter}{number}"

def plot_board(board_RR: torch.Tensor, title: str = "Board", png_filename: Optional[str] = None):
    """
    Plots an 8x8 board with values greater than 0.8 displayed in red text to two decimal places.
    
    Args:
    board_RR (torch.Tensor): A 2D tensor of shape (8, 8) with values from 0 to 1.
    title (str): Title of the plot.
    """
    assert board_RR.shape == (8, 8), "board_RR must be of shape 8x8"
    
    plt.imshow(board_RR, cmap='gray_r', interpolation='none', vmin=0, vmax=1)
    plt.colorbar()  # Adds a colorbar to help identify the values
    plt.grid(False)  # Turn off the grid lines
    plt.title(title)

    # Iterate over each element in the tensor to display values
    for i in range(board_RR.shape[0]):
        for j in range(board_RR.shape[1]):
            value = board_RR[i, j]
            if value > 0.8:
                # Display the value in red text at the corresponding position
                plt.text(j, i, f"{value:.0%}", color='red', ha='center', va='center', fontsize=12)

    if png_filename is not None:
        plt.savefig(png_filename)
    plt.show()

num_to_class = {0: "Black King", 1: "Black Queen", 2: "Black Rook", 3: "Black Bishop", 4: "Black Knight", 5: "Black Pawn",
                6: "Blank", 7: "White Pawn", 8: "White Knight", 9: "White Bishop", 10: "White Rook", 11: "White Queen", 12: "White King"}

In [ ]:
board_state_feature_labels_TFRRC = feature_labels['board_to_piece_masked_blank_and_initial_state']
print(f"Board state feature labels: {board_state_feature_labels_TFRRC.shape}")
threshold = 5

board_state_feature_labels_FRRC = board_state_feature_labels_TFRRC[threshold]
board_state_counts_F = einops.reduce(board_state_feature_labels_FRRC, "F R1 R2 C -> F", "sum")

max_features = 175
demo_idx = 0
for i in range(max_features):
    if board_state_counts_F[i] > 0:
        print(f"Feature {i} has {board_state_counts_F[i]} classified squares")
        demo_idx = i

demo_feature_labels_RRC = board_state_feature_labels_FRRC[demo_idx]
print(f"\nFeature {demo_idx} has {board_state_counts_F[demo_idx].sum().item()} classified squares")

classified_squares = torch.where(demo_feature_labels_RRC == 1)
print(f"Classified squares as tensors: {classified_squares}")

row, column, classes = classified_squares

print("\nClassified squares:")
for i in range(row.shape[0]):
    print(rc_to_square_notation(row[i].item(), column[i].item()), num_to_class[classes[i].item()])

In [ ]:
print(aggregation_results['on_count'].shape)

T, F = aggregation_results["on_count"].shape

print(f"For all {F} alive features over {T} thresholds, on_count is the number of times each feature is on above every threshold")

formatted_results = analysis.add_off_tracker(aggregation_results, custom_functions, analysis_device)

formatted_results = analysis.normalize_tracker(
    formatted_results,
    "on",
    custom_functions,
    analysis_device,
)

formatted_results = analysis.normalize_tracker(
    formatted_results,
    "off",
    custom_functions,
    analysis_device,
)

print(formatted_results["board_to_piece_masked_blank_and_initial_state"]['on_normalized'].shape)

board_results_TFRRC = formatted_results["board_to_piece_masked_blank_and_initial_state"]['on_normalized']

def plot_feature_board_states(board_results_TFRRC: torch.Tensor, feature_idx: int, threshold: int, piece_type: int):
    results_RRC = board_results_TFRRC[threshold, feature_idx]

    feature_on_count = formatted_results['on_count'][threshold, feature_idx]

    print(f"Feature {feature_idx} had {int(feature_on_count)} activations over threshold {(threshold * 10)}%")

    print(results_RRC.shape)
    results_RR = results_RRC[..., piece_type]
    print(results_RR)

    title = f"Average {num_to_class[piece_type]} activation for \nfeature {feature_idx} over threshold {(threshold * 10)}%"
    png_filename = f"feature_{feature_idx}_threshold_{threshold}_piece_{piece_type}.png"
    plot_board(results_RR, title, png_filename)

plot_feature_board_states(board_results_TFRRC, demo_idx, 0, 8)
plot_feature_board_states(board_results_TFRRC, demo_idx, 2, 8)
plot_feature_board_states(board_results_TFRRC, demo_idx, 2, 5)
plot_feature_board_states(board_results_TFRRC, demo_idx, 0, 3)
plot_feature_board_states(board_results_TFRRC, demo_idx, 5, 3)

In [ ]:
if config.analysis_on_cpu:
    aggregation_results = utils.to_device(aggregation_results, device)
    feature_labels = utils.to_device(feature_labels, device)
    misc_stats = utils.to_device(misc_stats, device)


In [ ]:

# Optionally, this can be sped up by
# config.board_reconstruction_n_inputs = 100

expected_reconstruction_output_location = expected_aggregation_output_location.replace(
    "results.pkl", "reconstruction.pkl"
)

if config.run_board_reconstruction:
    print("Testing board reconstruction")
    board_reconstruction_results = eval_board_reconstruction.test_board_reconstructions(
        custom_functions=custom_functions,
        autoencoder_path=autoencoder_path,
        feature_labels=feature_labels,
        output_file=expected_reconstruction_output_location,
        n_inputs=config.board_reconstruction_n_inputs,
        batch_size=config.batch_size,
        device=device,
        data=utils.to_device(test_data.copy(), device),
        othello=othello,
        print_results=False,
        save_results=config.save_results,
        precomputed=config.precompute,
    )
else:
    with open(expected_reconstruction_output_location, "rb") as f:
        board_reconstruction_results = pickle.load(f)
    board_reconstruction_results = utils.to_device(board_reconstruction_results, device)
